# Used Cars Price Prediction for UK-based Online Car Marketplace
--- 

# 1. Introduction
---
## 1.1 Business & Context Understanding

###     Business Problem
The client is an emerging online used car listing company, similar to platforms like OLX Autos, Carmudi, or Carsome.id. Their business model revolves around acquiring pre-owned vehicles in good condition at prices below market value, with the intention of reselling them on their platform at a higher price point.

### Business Model: 
The company strategically purchases used cars at favorable prices, aiming to maximize profit margins upon resale. The success of their business model depends on acquiring vehicles at a cost that allows for competitive pricing in the online marketplace while ensuring a reasonable profit.

### Business Problem:
The primary challenge faced by the company is the potential reduction in profit margins if they acquire used cars at prices higher than the market value. This emphasizes the need for a predictive model that can assist in estimating the appropriate purchase price range for pre-owned vehicles.
- Overpricing: If the purchase price is too high, the company may be forced to sell the vehicle at a price higher than the market value, resulting in a loss.
- Underpricing: If the purchase price is too low, the company may not be be able to purchase the vehicle at the price point, resulting in a missed opportunity.

### Business Success Criteria:
The success of the project is defined by achieving the following criteria:

- Accurate Price Prediction: Develop a machine learning model capable of accurately predicting the resale price of used cars based on relevant features.
- Profit Consideration: Ensure that the predicted price range allows for a reasonable profit margin, aligning with the company's business goals.
- Optimized Purchase Decisions: Empower agents purchasing used cars to make informed decisions by providing them with a predicted price range for each vehicle.




## 1.2 Proposed Solution: 

###  **Machine Learning Regression Model for Price Prediction:**
To address the business problem, the proposed solution involves creating a robust machine learning model for predicting the resale price of used cars. This model will take into account various features such as the make, model, mileage, year of manufacture, and any other relevant factors that influence the pricing of used cars.

The model's implementation aims to empower purchasing agents with a tool that can provide a reasonable estimate of the resale value of a potential acquisition. This ensures that the company can make well-informed decisions when acquiring used cars, optimizing the balance between competitive pricing and maintaining healthy profit margins.




###  **Model Evaluation:**
####  Evaluation Metrics:

*   **Mean Absolute Error (MAE):**
    
    *   MAE represents the average absolute difference between the predicted prices and the actual prices.
    *   Business Relation: A low MAE indicates that, on average, the model's predictions are close to the true prices. This is crucial for the business, as it directly aligns with the goal of accurate price prediction, helping purchasing agents make informed decisions.
*   **Mean Squared Error (MSE):**
    
    *   MSE calculates the average squared difference between predicted and actual prices.
    *   Business Relation: MSE gives more weight to large errors. A low MSE indicates that the model is effective in minimizing significant deviations, which is important for ensuring that extreme pricing errors are minimized, contributing to more consistent and reliable predictions.
*   **R-squared (R2):**
    
    *   R2 measures the proportion of the variance in the target variable explained by the model.
    *   Business Relation: A high R2 suggests that a significant portion of the variability in used car prices is captured by the model. This is important for business success as it indicates that the model is effectively leveraging the provided features to predict prices, contributing to better decision-making.




### **Model Interpretation:**
#### Feature Importance:
Feature Importance Analysis:
Interpret the importance of each feature in the model. Identify which features have the most significant impact on price predictions.
Business Relation: Understanding feature importance helps the business identify key factors influencing used car prices. This insight can guide strategic decisions, such as focusing on specific features during the car acquisition process.


#### Visualization:
Prediction vs. Actual Plots:
Visualize model predictions against actual prices, considering different subsets of the data if necessary.
Business Relation: Visualization helps stakeholders grasp the overall performance of the model. If the predicted prices align closely with actual prices, it instills confidence in the model's reliability and contributes to better decision-making.



## 1.3 Data Understanding

### About the Data
- Dataset contains listing of used cars in UK from several providers
- Scraped data of used cars listings. 100,000 listings, which have been separated into files corresponding to each car manufacturer.
- The cleaned data set contains information of price, transmission, mileage, fuel type, road tax, miles per gallon (mpg), and engine size. 


---